In [1]:
#libraries
import cv2
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from sklearn.neighbors import KDTree
import time
import argparse
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm
import zipfile
from skimage.color import rgb2hsv,rgb2lab

In [2]:
#read video file frame by frame, beginning and ending with a timestamp
def read_video_segments(video,start_frame,end_frame,resolution_width=200):
    resolution_height=int(round(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    with tqdm(total=end_frame-start_frame) as pbar: #init the progressbar,with max lenght of the given segment
        while(vid.isOpened()):
            # Capture frame-by-frame
            ret, frame = vid.read() # if ret is false, frame has no content
            if not ret:
                break
            # skip every "skip_frame"
            if vid_length>=start_frame:
                # resize the video to a different resolution
                frame=cv2.resize(frame,resolution)
                frames.append(frame) #add the individual frames to a list
                pbar.update() #update the progressbar
            if vid_length==end_frame:
                pbar.update()
                break
            vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    return frames

In [3]:
def extract_dominant_colors(frame_list):
    print(str(len(frame_list))+' frames to process.')
#     start=time.time()
    rgb_to_color=fn_rgb_to_color() #get the color dict 
    bins={} #a dict with an entry for each for histograms 
    for rgb in rgb_to_color: #init the dict with zeros for every key
        bins[rgb_to_color[rgb]]=0
        
    rgb_list=[] #create a list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)
    i = 0

    kdt = KDTree(rgb_list, leaf_size=30, metric='euclidean')  
    for image in tqdm(frame_list): #traverse the video
        #flatten the image to 1d 
        img = image.reshape((image.shape[0] * image.shape[1], 3))     
        nns = kdt.query(img, k=1, return_distance=False)
        for nn in nns:
            bins[rgb_to_color[rgb_list[nn[0]]]]+=1
        i+=1
#         end=time.time()
#         print('Finished '+str(i)+',time: '+str(end-start))
        norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1]#normalize the bins
        bins_norm={k:v/norm_factor for k,v in bins.items()}
    return bins_norm

In [4]:
def bins_to_df(bins,bin_threshold=5,colors_to_return=5):
    #create a dataframe, sorted descending by count
    bins_sorted=sorted(zip(list(bins.values()),list(bins.keys())),reverse=True)
    df=pd.DataFrame(bins_sorted,columns=['count','color'])
    df.set_index('color',inplace=True) #set the colors as the index of the dataframe
    bin_threshold=bin_threshold/100 #scale the percentage to 0-1
    df = df[df>bin_threshold].dropna() #kick bins from the dataframe with precentage lower than bin_threshold 
    return df.head(colors_to_return)#return the color_return highest bins, default 5, if less bins then
                                    #color_return are there return all

In [5]:
def fn_rgb_to_color(*path):
    if not ('no'):
        path=str(path)[2:-3] #to get rid of the of the *args things
        rgb_to_color = {}
        with open(path) as f:
            for line in f:
                #split lines at "::
                color, rgb = line.strip().split(':')
                #strip the rgb-string of the parenthesis, split it up a the commas,
                #cast them to int and put them into a tuples
                rgb_value=tuple(map(int,(rgb.strip('(').strip(')').split(','))))
                rgb_to_color[rgb_value] = color
    else:
        colors={'darkred':(139,0,0),
        'firebrick':(178,34,34),
        'crimson':(220,20,60),
        'red':(255,0,0),
        'tomato':(255,99,71),
        'salmon':(250,128,114),
        'darkorange':(255,140,0),
        'gold':(255,215,0),
        'darkkhaki':(189,183,107),
        'yellow':(255,255,0),
        'darkolivegreen':(85,107,47),
        'olivedrab':(107,142,35),
        'greenyellow':(173,255,47),
        'darkgreen':(0,100,0),
        'aquamarine':(127,255,212),
        'steelblue':(70,130,180),
        'skyblue':(135,206,235),
        'darkblue':(0,0,139),
        'blue':(0,0,255),
        'royalblue':(65,105,225),
        'purple':(128,0,128),
        'violet':(238,130,238),
        'deeppink':(255,20,147),
        'pink':(255,192,203),
        'antiquewhite':(250,235,215),
        'saddlebrown':(139,69,19),
        'sandybrown':(244,164,96),
        'ivory':(255,255,240),
        'dimgrey':(105,105,105),
        'grey':(28,128,128),
        'silver':(192,192,192),
        'lightgrey':(211,211,211),
        'black':(0,0,0),
        'white':(255,255,255),
        'darkcyan':(0,139,139),
        'cyan':(0,255,255),
        'green':(0,128,0),
        'khaki':(240,230,140),
        'goldenrod':(218,165,32),
        'orange':(255,165,0),
        'coral':(255,127,80),
        'magenta':(255,0,255),
        'wheat':(245,222,179),
        'skin':(255,224,189),
        'purple4':(147,112,219)}
        rgb_to_color={}
        for color in colors:
            rgb_to_color[colors[color]]=color
        #purple4 is median purple
        #skin is caucasian
    return rgb_to_color

In [6]:
def read_azp(azp_path):
    #extract the .azp-file to /tmp
    zip_ref = zipfile.ZipFile(azp_path)
    zip_ref.extractall('/tmp')
    #read the .xml-file
    tree = ET.parse('/tmp/content.xml')
    root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
    #traverse the .xml-file
    with open(args.output_path,'w') as file:
            if args.what_to_process=='scene':
                segment_list=[]
            for child in root[0].iter():
                if child.get('type')=='#Shot': #whenever a shot annotation is found, extract the timestamp from the xml
                    dominant_colors_list=[]
                    for child2 in child:
                        if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}millisecond-fragment':
                            end=int(child2.get('end'))/1000*25
                            begin=int(child2.get('begin'))/1000*25
                            if args.what_to_process=='scene': #if 'scene' is selected append the frames of the segments to a list
                                segment_list.append(read_video_segments(args.video_path,begin,end,args.resolution_width))
                            if args.what_to_process=='segment': #if 'segment' is selected run extract_dominant_colors on the segment
                                segment = read_video_segments(args.video_path,begin,end,args.resolution_width)
                                colors_df = bins_to_df(extract_dominant_colors(segment),args.bin_threshold,args.colors_to_return)
                                colors_list = [(color,perc) for color,perc in zip(colors_df.index.values,colors_df.values.tolist())]
                                print(begin,end,colors_list)
                                file.write((begin,end,colors_list)+'\n') #write the timestamp and the extracted colors to file
            if args.what_to_process=='scene': #if 'scene' is selected run extract_dominant_colors on the the list of segments
                colors_df = bins_to_df(extract_dominant_colors(segment_list),args.bin_threshold,args.colors_to_return)
                colors_list = [(color,perc) for color,perc in zip(colors_df.index.values,colors_df.values.tolist())]
                print(colors_list)
                file.write(colors_list+'\n') #write the extracted colors to file
            file.close()

In [ ]:
def read_azp2(azp_path):
    #extract the .azp-file to /tmp
    zip_ref = zipfile.ZipFile(azp_path)
    zip_ref.extractall('/tmp')
    #read the .xml-file
    tree = ET.parse('/tmp/content.xml')
    root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
    #traverse the .xml-file
    for child in root[0].iter():
        if child.get('type')=='#Shot': #whenever a shot annotation is found, extract the timestamp from the xml
            for child2 in child:
                if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}millisecond-fragment':
                    end=int(child2.get('end'))/1000*25
                    begin=int(child2.get('begin'))/1000*25

In [7]:
def azp_path(path):
    if path[-4:] == '.azp': #if the path is to a single file
        print('exactly')
#         read_azp(path)
    elif path[0][-4:] == '.azp': #if the path is to several files
        print('like')
        for azp_path in path:
            print(azp_path)
#             read_azp(azp_path)
    else: #else it is assumed the path points to a directory
        directory_content = os.listdir(path)
        azp_list=[]
        for elem in directory_content:
            if elem[-4:]=='.azp':
                if path[-1]=='/': #if the path ends with an '/', add the .azp-file
                    azp_list.append(path+elem)    
                else: #else, add a '/' and then the .azp-file
                    azp_list.append(path+'/'+elem)
        for azp_path in azp_list:
#             read_azp(azp_path)
            print(azp_path)
        print('planned')

In [8]:
def change_colorspace(frame_list,*target_colorspace):
    changed_frame_list=[]
    if target_colorspace=='HSV':
        for frame in frame_list:
            changed_frame_list.append(rgb2hsv(frame))
        return changed_frame_list
    if target_colorspace=='cie-lab':
        for frame in frame_list:
            changed_frame_list.append(rgb2lab(frame))
        return changed_frame_list
    else:
        return frame_list

In [9]:
#extract the .azp-file to /tmp
zip_ref = zipfile.ZipFile('CompanyMen_v1.0-split-012-Bobby_being_angry.azp')
zip_ref.extractall('/tmp')
#read the .xml-file
tree = ET.parse('/tmp/content.xml')
root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')

In [10]:
#traverse the .xml-file
for child in root[0].iter():
    if child.get('type')=='#ColourRange': #whenever a shot annotation is found, extract the timestamp from the xml
        dominant_colors_list=[]
        print(child.tail())
#         for child2 in child:
#             if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}content':
#                 print(child2.items())
#             print(child2)
        break
#             for child2 in child:
#                 if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}millisecond-fragment':

#                     end=int(child2.get('end'))/1000*25
#                     begin=int(child2.get('begin'))/1000*25
#                     if args.what_to_process=='scene': #if 'scene' is selected append the frames of the segments to a list
#                         segment_list.append(read_video_segments(args.video_path,begin,end,args.resolution_width))
#                     if args.what_to_process=='segment': #if 'segment' is selected run extract_dominant_colors on the segment
#                         segment = read_video_segments(args.video_path,begin,end,args.resolution_width)
#                         colors_df = bins_to_df(extract_dominant_colors(segment),args.bin_threshold,args.colors_to_return)
#                         colors_list = [(color,perc) for color,perc in zip(colors_df.index.values,colors_df.values.tolist())]
#                         print(begin,end,colors_list)
#                         file.write((begin,end,colors_list)+'\n') #write the timestamp and the extracted colors to file
#             file.close()

TypeError: 'NoneType' object is not callable

In [35]:
# 'fehlermaß'
colors_target=[['red','black','green','white'],['grey','orange','violet','crimson','blue','yellow']]
colors_pred=[['red','black','green','white'],['orange','violet','crimson','blue','yellow']]

In [36]:
def intersect(a, b):
    return list(set(a) & set(b))

In [37]:
len(colors_target[0])/len(list(set(colors_target[0]) & set(colors_pred[0])))

1.0

In [38]:
acc=[]
tl=0
pl=0
for target,pred in zip(colors_target,colors_pred):
#     acc.append(len(pred)/len(list(set(target) & set(pred))))
    tl+=len(target)
    pl+=len(list(set(target) & set(pred)))

In [39]:
# acc
pl/tl

0.9